### socketserver
简化了编写网络服务器的方式

---

该模块中包含有 5 个类
1. `class socketserver.BaseServer` 
    所有的服务器类的基类，基础的构建部分合适的服务器构建属性和操作
2. `class socketserver.TCPServer(server_address, RequestHandlerClass, bind_and_activate=True)`  
    利用 `TCP` 连接，在服务器和客户端之间提供流式的数据传输,如果属性 `bing_and_activate=True` 构造函数会自动的调用
    `server_bind`, `server_activate` 启动服务器，其他的传入属性自动继承自 `BaseServer` 类
3. `class socketserver.UDPServer(server_address, RequestHandlerClass, bind_and_activate=True)`  
    利用 `UDP` 连接，使用功能数据报的方式，离散的发送数据包，可能会丢失
4. `class socketserver.UnixStreamServer(server_address, RequestHandlerClass, bind_and_activate=True)`
5. `class socketserver.UnixDatagramServer(server_address, RequestHandlerClass, bind_and_activate=True)`  
    4,5类实际上参数和 `TCP` 等是一样的，但是实际上这两个类只能在 `Unix` 系统上运行，是 `Unix` 域的套接字
    
上述的 4 个主要的类的处理实际上是同步的处理，这意味着下一个请求只能在上一个请求完成之后才可以运行，但是这是不合适的，因为有可能请求非常的消耗时间(慢客户端,一直在传输文件，其他的客户端就没有办法连接上，这时候需要对服务器采用多线程)，解决这个的方案就是 **创建额外的线程或者是进程单独处理每一个连接请求操作**

异步的执行的话，可以考虑类 `ThreadingMixIn`, `ForkingMixIn`，当继承这两个类的时候，我们需要明确的指定当线程意外的终止的时候如何表现，`ThreadingMixIn` 类定义了一个属性 `daemon_threads`,这表明服务器是否应该等待线程终止。如果想要线程可以自主的运行就要设置这个属性，默认的情况下，Python中服务器会自动的等待所有的线程结束。如果某个客户端接受或处理数据花了很长时间，服务器无需等待处理完成，即可使用另外的线程和其他客户端进行通信。

**类不论使用的是什么协议，外部的属性和方法的调用和使用方式是一致的**

In [ ]:
+------------+
| BaseServer |
+------------+
      |
      v
+-----------+        +------------------+
| TCPServer |------->| UnixStreamServer |
+-----------+        +------------------+
      |
      v
+-----------+        +--------------------+
| UDPServer |------->| UnixDatagramServer |
+-----------+        +--------------------+

---
创建服务器的步骤

1. 创建一个**请求处理类**( `BaseRequestHandle` 类的子类，继承得到)，并且覆写 `handle()` 方法，该方法会处理得到的请求
2. 实例化一个服务器类，创给该类当前主机的地址和一个请求处理类，建议使用 `with` 使用服务器类，在 `with` 中会调用功能 `serve_forever`, `handle_request` 方法执行服务器去处理一个或者多个请求，如果使用 `with` 语句的话，就不需要使用 `server_close` 关闭服务器

---
其他类的用法
1. `class socketserver.ForkingMixIn` : 多进程(多路复用)套接字，异步处理客户端无需阻塞
2. `class socketserver.ThreadingMixIn` : 多线程套接字,对于共享变量请使用线程锁

上面的类可以用来辅助创建一个功能强大的异步服务器( `TCP` / `UDP` )  
下面的类只能支持 `POSIX` 平台

1. `class socketserver.ForkingTCPServer`
2. `class socketserver.ForkingUDPServer`
3. `class socketserver.ThreadingTCPServer`
4. `class socketserver.ThreadingUDPServer`

上面的类是使用之前的类预定义形成的

In [4]:
from socketserver import *

class ThreadingUDPServer(ThreadingMixIn, UDPServer):
    '''
    使用 UDPServer 可以省去手动创建服务器操作，如创建套接字、绑定地址和监听连接等
    使用 ThreadingMixIn 可以辅助创建一个强大的异步处理器
    混合继承，并且 ThreadingMixIn 必须放在第一个继承的位置，因为会重写 UDPServer 中的一个函数
    '''
    pass

为了启动服务，必须创建一个请求服务类,从 `BaseRequestHandle` 中继承并复写 `handle` 方法，可以将该类结合到不同的服务器类上以在不同的服务下执行相同的请求执行程序，对于不同的服务类型( `TCP` / `UDP` )，可以采用请求服务类的子类 `StreamRequestHandler`, `DatagramRequestHandler.`

---
服务器对象方法
1. `class socketserver.BaseServer(server_address, RequestHandlerClass)`  
    是所有的类的超类，定义的接口并不是所有的都有实现，有的需要在子类中实现，存储的参数有 `server_address`, `RequestHandlerClass`
    * `handle_request()`  
        处理请求，会在内部执行函数 `get_request()`, `verify_request()`, `process_request()`
        线程处理是存在有限定时间的，到时间没有请求出现会返回程序,或者说用户定义的 `handle` 出现异常返回的话，也会返回
    * `serve_forever(poll_interval=0.5)`
        处理请求直到服务器停止，`poll_interval` 是轮询的间隔
    * `server_address` : 服务器地址返回，如果是 Inet 协议，返回为 `(address, port)`
    * `timeout` : 请求时长限定
    * `socket_type` : socket协议的类型
    
2. `class socketserver.BaseRequestHandler` : 请求处理对象的超类  
    **每一个请求需要实例化一个请求处理对象并且实在服务器中自动实例化的，并且需要对其中的 `handle` 方法进行重写**
    * `setup()` : 
        在执行 `handle` 之前进行相应的初始化操作，默认是空
    * `handle()` :   
        执行服务一个请求的所有操作，默认是空，需要覆写
        可以访问请求处理对象的几个参数 `self.request`, `self.client_address`, `self.server`, 以便请求处理操作需要从这些参数中获取对应的信息  
        `self.request` 属性在不同的协议中还是不同的，在流式 `TCP` 协议中， `self.request` 是一个和远程的 `client` 连接的 `socket` 对象(**意味着，之前的 `socket.socket` 的所有接口可以使用包括读写等等**)，但是在 `UDP` 协议中， `self.request` 是一个 `(string, socket)`
    * `finish()`
        在 `handle()` 函数执行之后需要执行的清理操作，默认是空，如果 `setup()` 函数异常终止该函数不会被调用
        
3. `class socketserver.StreamRequestHandler` || `class socketserver.DatagramRequestHandler`  
    请求处理对象类的子类，并且对 `BaseRequstHandler` 类的 `setup(), finish()` 方法进行了封装和实现
    并且提供了新的封装参数 `self.rfile, self.wfile` 用来对套接字进行数据发送和数据获取
    其中的 `self.rfile` 其实本质上是对 `socket.recv` 多次调用(直到遇到换行符号停止接收)
    **需要注意的一点是，因为 `self.rfile` 本质上是对文件描述符的获取，所以客户端发送数据的时候必须要在末尾发送 `\n`,以确保我们使用 `self.rfile.readline()` 可以及时的读取到数据**
    
**请求处理对象可以从 `socketserver.BaseRequestHandler`, `socketserver.StreamRequestHandler`, `socketserver.DatagramRequestHandler` 三个中任意一个进行继承**

In [ ]:
# 对于没有使用异步的时候，服务器采用的是单线程，如果来到一个慢客户端，其他的客户端的请求会被阻塞
# 服务端
#!/usr/bin/python

import socketserver
import threading

class MyTCPHandle(socketserver.BaseRequestHandler):
    def handle(self):
        # self.request is the TCP socket for the remote client
        self.data = self.request.recv(1024)
        # print the client_address
        print("connected by", self.client_address, threading.current_thread())
        print(self.data)
        self.request.sendall(self.data)

class myTCPHandle(socketserver.StreamRequestHandler):
    def handle(self):
        while self.rfile.readline().strip():
            self.data = self.rfile.readline().strip()
            print("connected by", self.client_address, threading.current_thread(), self.data)
            self.wfile.write(self.data)

if __name__ == "__main__":
    host, port = '', 9998
    print(threading.current_thread())
    with socketserver.TCPServer((host, port), myTCPHandle) as server:
        server.serve_forever()

In [ ]:
# 慢客户端，会阻塞其他的客户端，需要对主机采用多线程处理
#!/usr/bin/python

import socket
import sys
import time

host, port = '', 9998
data = "lantian\n"

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    sock.connect((host, port))
    for i in range(1000):
        data = str(i) + '\n'
        sock.send(data.encode())
        time.sleep(1)
        print(i)

In [ ]:
# 多路复用的服务端,上述的实验是可以成功的将每一个客户端的连接请求分配在一个线程中去运行，不会阻塞(影响)其他的客户端正常工作
#!/usr/bin/python

import socketserver
import threading
import socket
import time

class MyTCPHandle(socketserver.BaseRequestHandler):
    def handle(self):
        # self.request is the TCP socket for the remote client
        p = self.request.recv(1024)
        while p:
            print(p)
            p = self.request.recv(1024)
            # print the client_address
        print("connected by", self.client_address, threading.current_thread())
        data = self.data.decode() + str(threading.current_thread)
        self.request.sendall(data.encode())

class myTCPHandle(socketserver.StreamRequestHandler):
    def handle(self):
        while self.rfile.readline().strip():
            self.data = self.rfile.readline().strip()
            print("connected by", self.client_address, threading.current_thread(), self.data)
            self.wfile.write(self.data)

class threadedtcpserver(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

def client(ip, port, message):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        data = message + '\n'
        sock.sendall(data.encode())
        response = sock.recv(1024).decode()
        print('response', response)
        time.sleep(10)

def client_slow(ip, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((ip, port))
        for i in range(1000):
            sock.send(str(i).encode())
            print(i)
            time.sleep(1)

if __name__ == "__main__":
    host, port = '', 9998
    
    server = threadedtcpserver((host, port), MyTCPHandle)
    with server:
        ip, port = server.server_address
        server.serve_forever()
        # slow_thread = threading.Thread(target = client_slow, args = (ip, port, ))
        # slow_thread.start()
        # client(ip, port, "fuck you 1")
        # client(ip, port, "fuck you 2")
        # client(ip, port, "fuck you 3")